In [1]:
import cobra
import cobrakbase
import pandas as pd

cobrakbase 0.2.5


In [2]:
df = pd.read_excel('fungi_central_metab.xlsx')

In [5]:
modelseed_local = cobrakbase.modelseed.from_local('../../../../ModelSEEDDatabase')

load: ../../../../ModelSEEDDatabase/Biochemistry/reactions.tsv
load: ../../../../ModelSEEDDatabase/Biochemistry/compounds.tsv
load: ../../../../ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: ../../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: ../../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: ../../../../ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [29]:
%run ../../../scripts/alot_of_tools_for_flux.py

In [208]:
df_rxn_cmp_def = pd.read_csv('fungi_core_rxns.tsv', sep='\t')
df_rxn_cmp_def = df_rxn_cmp_def.set_index('seed_id')
rxn_build_template = {}
for seed_id, d in df_rxn_cmp_def.iterrows():
    if seed_id not in rxn_build_template:
        rxn_build_template[seed_id] = []
    config = {}
    for k in d.keys():
        config[k] = d[k]
    rxn_build_template[seed_id].append(config)

In [209]:
bad = set()
def get_b_from_dir(d):
    lb = -1000
    ub = 1000
    if d == '=':
        pass
    elif d == '>':
        lb = 0
    elif d == '<':
        ub = 0
    else:
        print('???', d)
    return lb, ub

map_compounds = {}
map_reactions = {}
for row_id, d in df.iterrows():
    rxn_id = d['rxn id']
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    direction = d['directionality ']
    cmp = d['Compartment']
    if pd.isna(cmp):
        cmp = 'c'
    if pd.isna(direction):
        direction = rxn.data['direction']
    
    rxn_cmp_slots = set(map(lambda x : x[1], rxn.cstoichiometry))
    if len(rxn_cmp_slots) == 1 and len(cmp) == 1:
        cmp_config = {list(rxn_cmp_slots)[0] : cmp}
        #print(rxn_id, cmp_config)
        #print(rxn)
        crxn, cpd_cmp = modelseed_to_cobra_reaction(rxn, cmp_config, cmp)
        crxn['lower_bound'], crxn['upper_bound'] = get_b_from_dir(direction)
        for cpd_id in cpd_cmp:
            if cpd_id not in map_compounds:
                map_compounds[cpd_id] = set()
            map_compounds[cpd_id] |= cpd_cmp[cpd_id]
        map_reactions[crxn['id']] = crxn
    elif rxn_id in rxn_build_template:
        for cmp_config in rxn_build_template[rxn_id]:
            crxn, cpd_cmp = modelseed_to_cobra_reaction(rxn, cmp_config, cmp)
            crxn['lower_bound'], crxn['upper_bound'] = get_b_from_dir(direction)
            for cpd_id in cpd_cmp:
                if cpd_id not in map_compounds:
                    map_compounds[cpd_id] = set()
                map_compounds[cpd_id] |= cpd_cmp[cpd_id]
            map_reactions[crxn['id']] = crxn
    else:
        bad.add(rxn_id)
        #print(rxn_id, rxn_cmp_slots, cmp)
        #print(rxn)
        #print(rxn.build_reaction_string(True))
        pass

In [210]:
for rxn_id in bad:
    rxn = modelseed_local.get_seed_reaction(rxn_id)
    ss = "{}\t{}\t{}".format(rxn_id, rxn.data['equation'], rxn.data['definition'])
    print(ss)

rxn02527	(1) cpd00004[0] + (1) cpd00067[0] + (1) cpd01011[0] <=> (1) cpd00003[0] + (1) cpd03662[0]	(1) NADH[0] + (1) H+[0] + (1) Butanal[0] <=> (1) NAD[0] + (1) n-Butanol[0]
rxn00251	(1) cpd00009[0] + (1) cpd00032[0] + (1) cpd00067[0] => (1) cpd00001[0] + (1) cpd00011[0] + (1) cpd00061[0]	(1) Phosphate[0] + (1) Oxaloacetate[0] + (1) H+[0] => (1) H2O[0] + (1) CO2[0] + (1) Phosphoenolpyruvate[0]
rxn00250	(1) cpd00002[0] + (1) cpd00020[0] + (1) cpd00242[0] <=> (1) cpd00008[0] + (1) cpd00009[0] + (1) cpd00032[0] + (1) cpd00067[0]	(1) ATP[0] + (1) Pyruvate[0] + (1) H2CO3[0] <=> (1) ADP[0] + (1) Phosphate[0] + (1) Oxaloacetate[0] + (1) H+[0]
rxn00305	(1) cpd00032[0] + (1) cpd00038[0] => (1) cpd00011[0] + (1) cpd00031[0] + (1) cpd00061[0]	(1) Oxaloacetate[0] + (1) GTP[0] => (1) CO2[0] + (1) GDP[0] + (1) Phosphoenolpyruvate[0]
rxn00336	(1) cpd00260[0] <=> (1) cpd00036[0] + (1) cpd00040[0]	(1) Isocitrate[0] <=> (1) Succinate[0] + (1) Glyoxalate[0]
rxn00154	(1) cpd00003[0] + (1) cpd00010[0] + (1

In [211]:
all_compounds = []
all_reactions = []
for rxn_id in map_reactions:
    all_reactions.append(map_reactions[rxn_id])
for cpd_id in map_compounds:
    cpd = modelseed_local.get_seed_compound(cpd_id)
    for cmp in map_compounds[cpd_id]:
        cobra_cpd = modelseed_to_cobra_compound(cpd, cmp)
        cobra_cpd['name'] += " [{}]".format(cmp)
        all_compounds.append(cobra_cpd)

In [212]:
base_model = {
        'id' : 'core_fungi',
        'version' : '1',
        'compartments' : {'c' : 'cytosol', 'e' : 'extra'},
        'metabolites' : all_compounds,
        'reactions' : all_reactions,
        'genes' : [],
        'annotation' : {}
}

In [213]:
import json
with open('/Users/fliu/workspace/jupyter/core_fungi.json', 'w') as fh:
    fh.write(json.dumps(base_model))

In [144]:
seed_reaction = modelseed_local.get_seed_reaction('rxn00162')

In [156]:
print(seed_reaction.id, seed_reaction.build_reaction_string(True))

rxn00162 Oxaloacetate [0] + H+ [0] --> CO2 [0] + Pyruvate [0]


In [153]:
model_rxn, x = modelseed_to_cobra_reaction(seed_reaction, {'0' : 'c'}, 'c')
model_rxn

{'id': 'rxn00162_c',
 'name': 'rxn00162 [c]',
 'metabolites': {'cpd00032_c': -1.0,
  'cpd00067_c': -1.0,
  'cpd00011_c': 1.0,
  'cpd00020_c': 1.0},
 'lower_bound': 0,
 'upper_bound': 1000,
 'gene_reaction_rule': '',
 'annotation': {}}

In [30]:
crxn

{'id': 'rxn14428_k',
 'name': 'rxn14428 [k]',
 'metabolites': {'cpd00001_k': -1.0,
  'cpd00418_k': -1.0,
  'cpd18074_k': -1.0,
  'cpd00067_k': 1.0,
  'cpd00075_k': 1.0,
  'cpd18072_k': 1.0},
 'lower_bound': -1000,
 'upper_bound': 1000,
 'gene_reaction_rule': '',
 'annotation': {}}

In [21]:
print(rxn)

rxn14428: cpd00001 [0] + cpd00418 [0] + cpd18074 [0] <=> cpd00067 [0] + cpd00075 [0] + cpd18072 [0]


In [56]:
import cobrakbase
import copy

In [57]:
token = "DGXHUU34PUCT2V5JGUYA23P6TMQJM3FK"
kbase = cobrakbase.KBaseAPI(token)

In [58]:
template_v2 = kbase.get_object('GramNegModelTemplateV2', 'NewKBaseModelTemplates')


In [60]:
template_v2.keys()

dict_keys(['__VERSION__', 'biochemistry_ref', 'biomasses', 'compartments', 'compcompounds', 'complexes', 'compounds', 'domain', 'id', 'name', 'pathways', 'reactions', 'roles', 'subsystems', 'type'])

In [61]:
from cobrakbase.core.kbasefba import NewModelTemplate

In [62]:
template = NewModelTemplate(template_v2)

In [64]:
trxn = template.get_reaction('rxn00001_c')

In [66]:
trxn.data

{'GapfillDirection': '>',
 'base_cost': 3,
 'direction': '>',
 'forward_penalty': 0,
 'id': 'rxn00001_c',
 'maxforflux': 100,
 'maxrevflux': -100,
 'name': 'diphosphate phosphohydrolase',
 'reaction_ref': '489/6/6/reactions/id/rxn00001',
 'reverse_penalty': 0.4609,
 'templateReactionReagents': [{'coefficient': -1,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00001_c'},
  {'coefficient': -1,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00012_c'},
  {'coefficient': 2,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00009_c'},
  {'coefficient': 1,
   'templatecompcompound_ref': '~/compcompounds/id/cpd00067_c'}],
 'templatecompartment_ref': '~/compartments/id/c',
 'templatecomplex_refs': ['~/complexes/id/cpx01833',
  '~/complexes/id/cpx01834',
  '~/complexes/id/cpx01835',
  '~/complexes/id/cpx34574',
  '~/complexes/id/cpx52693'],
 'type': 'conditional'}

In [84]:
def metabolites(trxn, template):
    metabolites = {}
    for o in trxn.data['templateReactionReagents']:
        templatecompcompound_ref = o['templatecompcompound_ref']
        compcompound_id = templatecompcompound_ref.split('/')[-1]
        compcompound = list(filter(lambda x : x['id'] == compcompound_id, template.data['compcompounds']))[0]
        metabolites[compcompound_id] = o['coefficient']
    return metabolites
metab = metabolites(trxn, template)

In [77]:
rxn = modelseed_local.get_seed_reaction('rxn00001')

In [94]:
def extract_comp_config(rxn, trxn):
    comp_config = {}
    metab = metabolites(trxn, template)
    metab_c = dict(map(lambda x : ((x[0].split('_')[0], x[0].split('_')[1]), x[1]), metab.items()))
    print(rxn.cstoichiometry)
    print(metab_c)
    return metab_c, rxn.cstoichiometry
a, b = extract_comp_config(rxn, trxn)

{('cpd00001', '0'): -1.0, ('cpd00012', '0'): -1.0, ('cpd00009', '0'): 2.0, ('cpd00067', '0'): 1.0}
{('cpd00001', 'c'): -1, ('cpd00012', 'c'): -1, ('cpd00009', 'c'): 2, ('cpd00067', 'c'): 1}


In [118]:
def aaaa(a, b):
    a_c = get_compart(a)
    b_c = get_compart(b)
    if not len(a_c) == len(b_c):
        return None
    
    tt = list(itertools.product(get_compart(a), get_compart(b)))
    for t in tt:
        print(t)
    return True
aaaa(a, b)

('c', '0')


True

In [95]:
a

{('cpd00001', 'c'): -1,
 ('cpd00012', 'c'): -1,
 ('cpd00009', 'c'): 2,
 ('cpd00067', 'c'): 1}

In [96]:
b

{('cpd00001', '0'): -1.0,
 ('cpd00012', '0'): -1.0,
 ('cpd00009', '0'): 2.0,
 ('cpd00067', '0'): 1.0}

[('c', '0')]

In [97]:
for p in a:
    print(p)

('cpd00001', 'c')
('cpd00012', 'c')
('cpd00009', 'c')
('cpd00067', 'c')


In [98]:
def get_compart(o):
    return set(map(lambda x : x[1], o))

{'0'}

In [101]:
import itertools

In [133]:
list(itertools.combinations([0, 1, 2], 2))

[(0, 1), (0, 2), (1, 2)]

In [127]:
l1 = ['a', 'b']
l2 = [0, 2]
for p1 in l1:
    for p2 in l2:
        print(p1, p2)

a 0
a 2
b 0
b 2


In [143]:
list(itertools.product(*[['0', '1'], ['a', 'b']]))

[('0', 'a'), ('0', 'b'), ('1', 'a'), ('1', 'b')]

In [ ]:
('0', 'a')('1', 'b')  ('0', 'b')('1', 'a')